In [381]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import string
import re

In [382]:
df = pd.read_csv("datasets/data.csv")
df

,Unnamed: 0,statement,status
0,0,oh my gosh,Anxiety
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety
3,3,I've shifted my focus to something else but I'...,Anxiety
4,4,"I'm restless and restless, it's been a month n...",Anxiety
...,...,...,...
53038,53038,Nobody takes me seriously I’ve (24M) dealt wit...,Anxiety
53039,53039,"selfishness ""I don't feel very good, it's lik...",Anxiety
53040,53040,Is there any way to sleep better? I can't slee...,Anxiety
53041,53041,"Public speaking tips? Hi, all. I have to give ...",Anxiety


In [383]:
df = df[(df.status == 'Normal') | (df.status == 'Depression')]
df.reset_index()

,index,Unnamed: 0,statement,status
0,733,733,"Gr gr dreaming of ex crush to be my game, God",Normal
1,734,734,wkwkwk what a joke,Normal
2,735,735,Leaves are also standby in front of the PC ......,Normal
3,736,736,Thank God even though it's just a ride through,Normal
4,737,737,wedding teaser concept using the song day6 - o...,Normal
...,...,...,...,...
31750,48283,48283,"solutions So, when I have my real bad downs th...",Depression
31751,48284,48284,I’m pretty sure I’m depressed.. Ive been feeli...,Depression
31752,48285,48285,I can’t be bothered anymore. I’m a 22 year old...,Depression
31753,48286,48286,Depression feels like you’re drowning and ther...,Depression


In [384]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df.reset_index(drop=True,inplace=True)

C:\Users\Sanju\AppData\Local\Temp\ipykernel_24920\600921598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Unnamed: 0'],axis=1,inplace=True)


In [385]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31134 entries, 0 to 31527
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  31134 non-null  object
 1   status     31134 non-null  object
dtypes: object(2)
memory usage: 729.7+ KB


C:\Users\Sanju\AppData\Local\Temp\ipykernel_24920\2784204723.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
C:\Users\Sanju\AppData\Local\Temp\ipykernel_24920\2784204723.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [386]:
df['status'] = df['status'].apply(lambda x:1 if x == 'Normal' else 0)

C:\Users\Sanju\AppData\Local\Temp\ipykernel_24920\1983159667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['status'] = df['status'].apply(lambda x:1 if x == 'Normal' else 0)


In [387]:
train_x,test_x = df.statement[:25000],df.statement[25000:]
train_y,test_y = df.status[:25000],df.status[25000:]

In [388]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sanju\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sanju\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [389]:
def process_tweet(tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # Remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # Remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # Remove hyperlinks    
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)
    # Remove hashtags (only the hash # sign from the word)
    tweet = re.sub(r'#', '', tweet)
    # Tokenize the tweet
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    # Remove stopwords and punctuation, and apply stemming
    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_english and word not in string.punctuation):
            stem_word = stemmer.stem(word)  # Stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [390]:
train_x = train_x.apply(process_tweet)
test_x = test_x.apply(process_tweet)

In [391]:
# Build frequency dictionary
def build_freqs(tweets, ys):
    yslist = np.squeeze(ys).tolist()
    freqs = {}
    for y, tweet in zip(yslist, tweets):
        for word in tweet:
            pair = (word, y)
            if pair in freqs:
                freqs[pair] += 1
            else:
                freqs[pair] = 1
    return freqs

In [392]:
freqs = build_freqs(train_x,train_y)

In [393]:
# Extract features for logistic regression
def extract_features(tweet, freqs):
    x = np.zeros(3)
    x[0] = 1
    for word in tweet:
        x[1] += freqs.get((word, 1.0), 0)
        x[2] += freqs.get((word, 0.0), 0)
    return x

In [394]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [395]:
def gradient_descent(X, Y, theta, iterations, alpha):
    m = X.shape[0]
    for i in range(iterations):
        z = np.dot(X, theta)
        h = sigmoid(z)
        h = np.clip(h, 1e-15, 1 - 1e-15)  # Avoid log(0) errors
        J = (-1 / m) * (np.dot(Y.T, np.log(h)) + np.dot((1 - Y).T, np.log(1 - h)))
        theta = theta - (alpha / m) * np.dot(X.T, (h - Y))
        if i % 10 == 0:
            print(f"Iteration {i}, Cost: {J}")
    return J[0], theta

In [396]:
# Normalize and reshape the data
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :] = extract_features(train_x.iloc[i], freqs)

Y = train_y.values.reshape(-1, 1)  # Reshape Y to (25000, 1)
X = (X - X.min()) / (X.max() - X.min())  # Normalize features

In [397]:
# Train the logistic regression model
J, theta = gradient_descent(X, Y, np.zeros((3, 1)), 100, 0.0001)

Iteration 0, Cost: [[0.69314718]]
Iteration 10, Cost: [[0.69314705]]
Iteration 20, Cost: [[0.69314691]]
Iteration 30, Cost: [[0.69314678]]
Iteration 40, Cost: [[0.69314664]]
Iteration 50, Cost: [[0.69314651]]
Iteration 60, Cost: [[0.69314637]]
Iteration 70, Cost: [[0.69314624]]
Iteration 80, Cost: [[0.6931461]]
Iteration 90, Cost: [[0.69314597]]


In [398]:
# Predict for a single tweet
def predict_tweet(tweet, theta, freqs):
    x = extract_features(tweet, freqs)
    y_pred = sigmoid(np.dot(x, theta))
    return y_pred

In [399]:
# Test logistic regression accuracy
def test_logistic_regression(test_x, test_y, freqs, theta):
    y_hat = []
    for tweet in test_x:
        tweet_tokens = process_tweet(tweet)  # Process tweet into list of tokens (once)
        y_pred = predict_tweet(tweet_tokens, freqs, theta)  # Pass processed tokens
        if y_pred > 0.5:
            y_hat.append(1)
        else:
            y_hat.append(0)

    accuracy = (np.array(y_hat) == np.squeeze(test_y)).sum() / len(test_x)
    return accuracy

In [400]:
# Run the test and print accuracy
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

TypeError: expected string or bytes-like object, got 'list'